In [1]:
import requests
import pandas as pd

seasons = pd.read_csv('data/seasons.csv')
eur_seasons = pd.read_csv('data/european_seasons.csv')

In [5]:
eur_seasons.head()

,Unnamed: 0,league_id,name,country,year,players,statistics_fixtures,current,predictions,odds
0,0,39,Premier League,England,2010,True,False,False,True,False
1,1,39,Premier League,England,2011,True,False,False,True,False
2,2,39,Premier League,England,2012,True,False,False,True,False
3,3,39,Premier League,England,2013,True,False,False,True,False
4,4,39,Premier League,England,2014,True,True,False,True,False


In [18]:
df, r = get_data('fixtures',{'league':39, 'season':2023})
fix = [encode_data(row) for row in df['response']]

requests before reaching limit 99


In [191]:
fdf = pd.DataFrame(fix)
fdf['fixture_date'] = pd.to_datetime(fdf['fixture_date']).dt.date

In [192]:
home = fdf[[
    'fixture_date', 
    'teams_home_id', 
    'goals_home',
    'goals_away',
    'teams_home_winner', 
    'league_round'
    ]].rename(columns={
    'teams_home_id':'team_id',
    'goals_home':'goals_scored',
    'goals_away':'goals_lost',
    'teams_home_winner':'points'
    })
away = fdf[[
    'fixture_date', 
    'teams_away_id', 
    'goals_away',
    'goals_home',
    'teams_away_winner', 
    'league_round'
    ]].rename(columns={
    'teams_away_id':'team_id', 
    'goals_away':'goals_scored',
    'goals_home':'goals_lost',
    'teams_away_winner':'points'
    })

In [193]:
total = pd.concat([home, away])
total = total.sort_values(by='fixture_date')
total['total_goals_scored'] = total[['fixture_date','team_id','goals_scored']].groupby('team_id')['goals_scored'].cumsum()
total['total_goals_lost'] = total[['fixture_date','team_id','goals_lost']].groupby('team_id')['goals_lost'].cumsum()

In [194]:
def logic(x):
    if x==True:
        return 3
    elif x==False:
        return 0
    else:
        return 1

In [195]:
total = pd.concat([home, away])
total = total.sort_values(by='fixture_date')
total['total_goals_scored'] = total[['fixture_date','team_id','goals_scored']].groupby('team_id')['goals_scored'].cumsum()
total['total_goals_lost'] = total[['fixture_date','team_id','goals_lost']].groupby('team_id')['goals_lost'].cumsum()

total = total.sort_values(by='fixture_date')
total['points'] = total['points'].apply(logic)
total['total_points'] = total[['fixture_date', 'team_id', 'league_round', 'points']].groupby('team_id')['points'].cumsum()

total.sort_values(by=['league_round','total_points','total_goals_scored','fixture_date'], ascending=[True,False,False,True])
total['standings'] = total.groupby('league_round')['total_points'].rank(method='min', ascending=False)
total['standings'] = total['standings'].astype(int)

total = total.sort_values(by=['team_id','fixture_date'])
total['points_last_5_matches'] = total.groupby('team_id')['points'].rolling(window=5, min_periods=1).sum().reset_index(level=0, drop=True)
total['points_last_5_matches'] = total['points_last_5_matches'].fillna(0)
total['points_last_5_matches'] = total['points_last_5_matches'].astype(int)

In [196]:
total.tail(5)

,fixture_date,team_id,goals_scored,goals_lost,points,league_round,total_goals_scored,total_goals_lost,total_points,standings,points_last_5_matches
335,2024-04-20,1359,1,5,0,Regular Season - 34,47,75,25,18,3
349,2024-04-27,1359,1,2,0,Regular Season - 35,48,77,25,18,3
357,2024-05-03,1359,1,1,1,Regular Season - 36,49,78,26,18,4
368,2024-05-11,1359,1,3,0,Regular Season - 37,50,81,26,18,1
377,2024-05-19,1359,2,4,0,Regular Season - 38,52,85,26,18,1


In [197]:
#total_points = total_points.merge(total_goals, on=['fixture_date','team_id']).drop(columns=['points','goals'])

In [200]:
fdf = fdf.merge(total[[
    'fixture_date',
    'team_id',
    'total_goals_scored',
    'total_goals_lost', 
    'points', 
    'total_points', 
    'standings',
    'points_last_5_matches'
    ]], left_on = [
        'fixture_date',
        'teams_home_id'
        ],right_on = [
        'fixture_date',
        'team_id'
        ], how='left'
        ).rename(columns={
            'total_goals_scored':'teams_home_total_goals_scored',
            'total_goals_lost':'teams_home_total_goals_lost',
            'points':'teams_home_points',
            'total_points':'teams_home_total_points',
            'standings':'teams_home_standings',
            'points_last_5_matches':'teams_home_last_five_matches_points'
        }).drop(columns='team_id')

fdf = fdf.merge(total[[
    'fixture_date',
    'team_id',
    'total_goals_scored',
    'total_goals_lost', 
    'points', 
    'total_points', 
    'standings',
    'points_last_5_matches'
    ]], left_on = [
        'fixture_date',
        'teams_away_id'
        ],right_on = [
        'fixture_date',
        'team_id'
        ], how='left'
        ).rename(columns={
            'total_goals_scored':'teams_away_total_goals_scored',
            'total_goals_lost':'teams_away_total_goals_lost',
            'points':'teams_away_points',
            'total_points':'teams_away_total_points',
            'standings':'teams_away_standings',
            'points_last_5_matches':'teams_away_last_five_matches_points'
        }).drop(columns='team_id')

In [82]:
fdf = fdf.merge(total_goals[['fixture_date','team_id','total_goals']], left_on=['fixture_date','teams_home_id'], right_on=['fixture_date','team_id'], how='left').drop(columns=['team_id']).rename(columns={'total_goals':'goals_home_total'})
fdf = fdf.merge(total_goals[['fixture_date','team_id','total_goals']], left_on=['fixture_date','teams_away_id'], right_on=['fixture_date','team_id'], how='left').drop(columns=['team_id']).rename(columns={'total_goals':'goals_away_total'})
fdf = fdf.merge(total_points[['fixture_date','team_id','total_points']], left_on=['fixture_date','teams_home_id'], right_on=['fixture_date','team_id'], how='left').drop(columns=['team_id']).rename(columns={'total_points':'points_home_total'})
fdf = fdf.merge(total_points[['fixture_date','team_id','total_points']], left_on=['fixture_date','teams_away_id'], right_on=['fixture_date','team_id'], how='left').drop(columns=['team_id']).rename(columns={'total_points':'points_away_total'})

0

In [17]:
def get_data(endpoint, params):
    
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [20]:
def encode_data(data_dict, parent_key = '', sep= '_'):
    encoded = []
    for key, val in data_dict.items():
        new_key = f'{parent_key}{sep}{key}' if parent_key else key
        if isinstance(val, dict):
            encoded.extend(encode_data(val, new_key, sep=sep).items())
        elif isinstance(val, list):
            if val:
                if all(isinstance(i, dict) for i in val):
                    for k, v in enumerate(val):
                        v_key = f'{new_key}{sep}{k}'
                        encoded.extend(encode_data(v, v_key, sep=sep).items())
                else:
                    encoded.append((new_key, val))
            else:
                encoded.append((new_key, []))
        else:
            encoded.append((new_key, val))
    return dict(encoded)

In [42]:
def encode_fix_stats(data, fixture_id):
    encoded = []
    encoded.append(('fixture_id', fixture_id))
    for key, val in data.items():
        if key =='team':
            encoded.append((key+'_id', val['id']))
        else:
            for el in val:
                encoded.append((el['type'].lower().replace(' ', '_').replace('%', 'percentage'), el['value']))
    return dict(encoded)

In [43]:
def data_to_sql(table_name, df, db_params, conflict_columns, update_columns):
    conn = None
    cur = None
    try:
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        #insert data into tables
        if len(conflict_columns) == 0:
            insert_query = """
                INSERT INTO {} ({})
                VALUES ({})
            """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)))
        else:
            if update_columns:
                update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_columns])
                insert_query = """
                    INSERT INTO {} ({})
                    VALUES ({})
                    ON CONFLICT ({}) DO UPDATE SET {}
                """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)), ','.join(conflict_columns), update_set)
            else:
                insert_query = """
                    INSERT INTO {} ({})
                    VALUES ({})
                    ON CONFLICT ({}) DO NOTHING
                """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)), ','.join(conflict_columns))

        # Execute insert query for each row
        for row in df.itertuples(index=False, name=None):
            cur.execute(insert_query, row)
        
        # Commit the transaction
        conn.commit()       
    except Exception as e:
        print(f"Error: {e}")
        if conn:
            conn.rollback()

    finally:
        if conn is not None:
            # Close the cursor and connection
            cur.close()
        if cur is not None:
            conn.close()

In [44]:
def get_last_date(db_params, today):
    get_data_query = '''
    SELECT fixture_id
    FROM fixtures 
    WHERE fixture_status_short != 'FT' and fixture_status_short != 'WO' and fixture_status_short != 'AET' and fixture_status_short != 'PEN' and fixture_status_short != 'CANC' and fixture_date < today
    '''
    res = []
    conn = None
    cur = None
    try:
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        cur.execute(get_data_query)
        
        res = [row[0] for row in cur.fetchall()]
    except Exception as e:
        print(f"Error: {e}")
        if last_row is not None:
            print(f"Last row loaded before the error occurred: {last_row}")
    finally:
        if conn is not None:
            # Close the cursor and connection
            cur.close()
        if cur is not None:
            conn.close()
        return res

In [45]:
def get_teams(teams, df):
    for row in df:
        to_add_1 = {'league': row['league']['id'], 'season': row['league']['season'], 'team': row['teams']['home']['id']}
        to_add_2 = {'league': row['league']['id'], 'season': row['league']['season'], 'team': row['teams']['away']['id']}
        teams.extend([to_add_1, to_add_2])
    return teams

##### Get fixtures and their stats played during collecting historical data

In [ ]:
import pandas as pd
from datetime import date

remaining = 100
fixtures_data = []
fixture_stats_data = []
teams_to_update = []
s3 = boto3.client('s3')

bucket_name = 'sport-storage'
matches_key = 'total_fixs.json'
teams_to_u = 'teams_to_update.csv'

matches_response = s3.get_object(Bucket = bucket_name, Key = matches_key)
teams_to_upd = s3.get_object(Bucket = bucket_name, Key = teams_to_upd)
total_fixs = pickle.loads(matches_response['Body'].read())
teams_update = pd.read_csv(io.BytesIO(missed['Body'].read()))

if len(total_fixs) == 0:
    today = date.today()
    fixtures_to_find = get_last_date(db_params, today)
    total_fixs = fixtures_to_find
else:
    fixtures_to_find = total_fixs
    
while (remaining > 0) & (len(fixtures_to_find) > 0):
    
    ids = ''
    sep = '-'
    # get fixtures ids to find
    if len(fixtures_to_find) > 20:
        for i in range(0,20):
            ids = f'{ids}{sep}{fixtures_to_find[i]}' if ids else fixtures_to_find[i]
        fixtures_to_find = fixtures_to_find[20:]
    
    elif len(fixtures_to_find) > 0:
        for i in fixtures_to_find:
            ids = f'{ids}{sep}{i}' if ids else i
        fixtures_to_find = []
    
    #get fixtures
    if ids:      
        params = {'ids': ids}
        
        df, remaining = get_data('fixtures', params)
        if len(df['response']) > 0:
            fixtures_data.extend(encode_data(row) for row in df['response'])
            teams_to_update = get_teams(teams_to_update, df['response']))
        
while (remaining > 0) & (len(total_fixs) > 0):

    params = {'fixture': total_fixs[0]}
    df, remaining = get_data('fixtures/statistics', params)
    fixture_stats_data.extend(encode_fix_stats(df['response'], total_fixs[0]))
    total_fixs.pop(0)

if len(fixtures_data) > 0:
    fixtures_df = pd.DataFrame(fixtures_data)
    fixtures_df = fixtures_df.drop(columns = {
        'league_name',
        'league_country',
        'league_logo',
        'league_flag',
        'fixture_venue_city',
        'fixture_venue_name',
        'teams_away_logo',
        'teams_away_name',
        'teams_home_logo',
        'teams_home_name'})
    conflict_col = ['fixture_id']
    update_col = list(fixtures_df.columns)
    data_to_sql('fixtures', fixtures_df, db_params, conflict_col, update_col)
    
if len(fixture_stats_data) > 0:
    fixture_stats_df = pd.DataFrame(fixture_stats_data)
    conflict_col = ['fixture_id', 'team_id']
    update_col = list(fixture_stats_df.columns)
    data_to_sql('fixture_statistics', fixture_stats_df, db_params, conflict_col, update_col)

teams_update_df = pd.DataFrame(teams_to_update).drop_duplicates()
teams_update = pd.concat(teams_update_df, teams_update)

csv_buffer = io.StringIO()
teams_update.to_csv(csv_buffer, index=False)
s3.put_object(Bucket = bucket_name, Key = object_key1, Body=csv_buffer.getvalue().encode())

json_data = json.dump(total_fixs)
s3.put_object(Bucket = bucket_name, Key = teams_to_upd, Body=json_data)

if len(total_fixs) == 0:
    print('fixture data is actual')
## update team stats and fixs stats 
## get fixs to update from list total_fix

##### Get team stats before starting predicting

In [ ]:
import pandas as pd
from datetime import date

remaining = 100
total_team_stats_data = []
s3 = boto3.client('s3')

bucket_name = 'sport-storage'
teams_to_u = 'teams_to_update.csv'

teams_to_upd = s3.get_object(Bucket = bucket_name, Key = teams_to_upd)
teams_update = pd.read_csv(io.BytesIO(missed['Body'].read()))

to_find == 'teams/statistics'

if len(teams_update)>0:
    done = False
else:
    done = True
    
while (done==False)&(remaining > 0):
    team = team_update.loc[0]
    params = {'league': team['league'],
              'season': team['season'],
              'team': team['team']
             }
    endpoint = to_find
    data, remaining_req = get_data(endpoint, params)
    total_team_stats_data.append(encode_data(data['response']))
        
        # drop team that data was already collected for
    if len(teams_update) > 0:
        team_update = team_update.loc[1:]

        # if data for all seasons were collected quit loop
    else:
        done = True
                    
    remaining = int(remaining_req)
    print(remaining)
    # sleep cause there can be done only 10 requests per minute
    time.sleep(7)
db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}
if len(total_team_stats_data) > 0:
    team_stats_df = pd.DataFrame(total_team_stats_data)
    team_stats_df = team_stats_df.drop(columns = {
        'league_name', 
        'league_country', 
        'league_logo', 
        'league_flag', 
        'team_name', 
        'team_logo',
        'lineups'})
    team_stats_df = team_stats_df.fillna(0)
    conflict_col = ['league_id', 'team_id']
    update_col = list(team_stats_df.columns)
    team_stats_df.columns = team_stats_df.columns.str.replace('-','_')
    data_to_sql('team_stats', team_stats_df, db_params, conflict_col, update_col)
    
csv_buffer = io.StringIO()
teams_update.to_csv(csv_buffer, index=False)
s3.put_object(Bucket = bucket_name, Key = object_key1, Body=csv_buffer.getvalue().encode())

In [ ]:
''' 
1. get all historical data,
2. then find last results date and if season is current get missing results, team stats,

7 days matches

3. start collecting data about upcoming matches in next 7 days,
4. create predictions model,
5. create table for predictions and actual results,
6. after that 7 days get real result and save them in fixtures and preds table,
7. decide whats better add results to team stats table or collect latest data about team stats (??),
8. use preds and real results for models' training,
9. create flask app to display preds,
10. get upcoming fixtures odds and compare them to preds,
11. filter odds worth betting,
12. display that bets on flask api
13. display statistics for that bets,
'''
